In [ ]:
import numpy as np
import glob
import os
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from sklearn.model_selection import StratifiedKFold, cross_val_score, cross_validate
from google.colab import drive
from sklearn.model_selection import LeaveOneOut, GridSearchCV

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def apply_global_signal_regression(timeseries: np.ndarray) -> np.ndarray:
    global_signal = np.mean(timeseries, axis=1, keepdims=True)
    return timeseries - global_signal

def build_correlation_matrix(timeseries: np.ndarray) -> np.ndarray:
    """Compute correlation matrix while handling NaNs and checking constant regions."""
    timeseries = np.nan_to_num(timeseries, nan=0.0)
    variances = np.var(timeseries, axis=0)
    zero_var_regions = np.where(variances == 0)[0]
    if len(zero_var_regions) > 0:
        return None
    corr_matrix = np.corrcoef(timeseries, rowvar=False)
    return np.clip(np.nan_to_num(corr_matrix, nan=0.0, posinf=1.0, neginf=-1.0), -1.0, 1.0)

def flatten_upper_triangle(corr_matrix: np.ndarray) -> np.ndarray:
    """Extract the upper triangle (excluding diagonal) as a feature vector."""
    triu_indices = np.triu_indices(corr_matrix.shape[0], k=1)
    return corr_matrix[triu_indices]

def load_timeseries_data(ad_dir, cn_dir):
    ad_files = sorted(glob.glob(os.path.join(ad_dir, "*.txt")))
    cn_files = sorted(glob.glob(os.path.join(cn_dir, "*.txt")))

    X, y = [], []

    for fpath in ad_files:
        data_array = np.loadtxt(fpath)
        data_array = apply_global_signal_regression(data_array)
        corr_mat = build_correlation_matrix(data_array)

        if corr_mat is not None:
            X.append(flatten_upper_triangle(corr_mat))
            y.append(1)

    for fpath in cn_files:
        data_array = np.loadtxt(fpath)
        corr_mat = build_correlation_matrix(data_array)
        if corr_mat is not None:
            X.append(flatten_upper_triangle(corr_mat))
            y.append(0)

    return np.array(X), np.array(y)

In [ ]:
def evaluate_model_with_cv(X, y, n_splits=5):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    svm = SVC(kernel="rbf", C=1.0, gamma="scale")

    scoring = ['accuracy', 'precision', 'recall', 'f1']
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)
    scores = cross_validate(svm, X_scaled, y, cv=skf, scoring=scoring, return_train_score=False)

    results = {metric: (np.mean(scores[f'test_{metric}']), np.std(scores[f'test_{metric}']))
               for metric in scoring}

    sensitivities = []
    specificities = []

    for train_idx, test_idx in skf.split(X_scaled, y):
        svm.fit(X_scaled[train_idx], y[train_idx])
        y_pred = svm.predict(X_scaled[test_idx])
        cm = confusion_matrix(y[test_idx], y_pred)
        tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivities.append(sensitivity)
        specificities.append(specificity)

    results["Sensitivity"] = (np.mean(sensitivities), np.std(sensitivities))
    results["Specificity"] = (np.mean(specificities), np.std(specificities))

    return results

def evaluate_with_loocv(X, y):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    svm = SVC(kernel="rbf", C=1.0, gamma="scale")
    loo = LeaveOneOut()

    y_true, y_pred = [], []
    for train_idx, test_idx in loo.split(X_scaled, y):
        svm.fit(X_scaled[train_idx], y[train_idx])
        pred = svm.predict(X_scaled[test_idx])
        y_true.append(y[test_idx][0])
        y_pred.append(pred[0])

    cm = confusion_matrix(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
    sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
    specificity = tn / (tn + fp) if (tn + fp) > 0 else 0

    results = {
        "accuracy": (accuracy_score(y_true, y_pred), 0.0),
        "precision": (precision_score(y_true, y_pred, zero_division=0), 0.0),
        "recall": (recall_score(y_true, y_pred, zero_division=0), 0.0),
        "f1": (f1_score(y_true, y_pred, zero_division=0), 0.0),
        "Sensitivity": (sensitivity, 0.0),
        "Specificity": (specificity, 0.0)
    }
    return results

def evaluate_with_nested_cv(X, y, outer_folds=5, inner_folds=3):
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    param_grid = {
        'C': [0.1, 1.0, 10.0],
        'gamma': ['scale', 0.1, 0.01]
    }

    outer_cv = StratifiedKFold(n_splits=outer_folds, shuffle=True, random_state=42)
    inner_cv = StratifiedKFold(n_splits=inner_folds, shuffle=True, random_state=42)

    accuracies, precisions, recalls, f1s, sensitivities, specificities = [], [], [], [], [], []

    for train_idx, test_idx in outer_cv.split(X_scaled, y):
        X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]

        grid = GridSearchCV(SVC(kernel="rbf"), param_grid, cv=inner_cv, scoring='accuracy')
        grid.fit(X_train, y_train)

        best_model = grid.best_estimator_
        y_pred = best_model.predict(X_test)

        accuracies.append(accuracy_score(y_test, y_pred))
        precisions.append(precision_score(y_test, y_pred, zero_division=0))
        recalls.append(recall_score(y_test, y_pred, zero_division=0))
        f1s.append(f1_score(y_test, y_pred, zero_division=0))

        cm = confusion_matrix(y_test, y_pred)
        tn, fp, fn, tp = cm.ravel() if cm.shape == (2, 2) else (0, 0, 0, 0)
        sens = tp / (tp + fn) if (tp + fn) > 0 else 0
        spec = tn / (tn + fp) if (tn + fp) > 0 else 0
        sensitivities.append(sens)
        specificities.append(spec)

    results = {
        "accuracy": (np.mean(accuracies), np.std(accuracies)),
        "precision": (np.mean(precisions), np.std(precisions)),
        "recall": (np.mean(recalls), np.std(recalls)),
        "f1": (np.mean(f1s), np.std(f1s)),
        "Sensitivity": (np.mean(sensitivities), np.std(sensitivities)),
        "Specificity": (np.mean(specificities), np.std(specificities))
    }

    return results

In [ ]:
def print_formatted_results(results_dict):
    print("\nFinal SVM Classification Results:")
    print("------------------------------------------------------")
    print(f"{'Metric':<20} {'Mean':<10} {'Std Dev'}")
    print("------------------------------------------------------")
    rename = {
        "accuracy": "Accuracy",
        "precision": "Precision (AD)",
        "recall": "Recall (AD)",
        "f1": "F1-score (AD)",
        "Sensitivity": "Sensitivity",
        "Specificity": "Specificity"
    }
    for metric, (mean, std) in results_dict.items():
        label = rename.get(metric, metric)
        print(f"{label:<20} {mean:.2f} ± {std:.2f}")
    print("------------------------------------------------------")

**5-Fold Cross-Validation**

In [ ]:
## AAL90
AD_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_AAL90/AD"
CN_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_AAL90/CN"

X, y = load_timeseries_data(AD_DIR, CN_DIR)
results = evaluate_model_with_cv(X, y, n_splits=5)
print_formatted_results(results)


Final SVM Classification Results:
------------------------------------------------------
Metric               Mean       Std Dev
------------------------------------------------------
Accuracy             0.96 ± 0.08
Precision (AD)       0.94 ± 0.11
Recall (AD)          1.00 ± 0.00
F1-score (AD)        0.97 ± 0.07
Sensitivity          1.00 ± 0.00
Specificity          0.92 ± 0.16
------------------------------------------------------


In [ ]:
## Schaefer200
AD_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_Schaefer200/AD"
CN_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_Schaefer200/CN"
X, y = load_timeseries_data(AD_DIR, CN_DIR)
results = evaluate_model_with_cv(X, y, n_splits=5)
print_formatted_results(results)


Final SVM Classification Results:
------------------------------------------------------
Metric               Mean       Std Dev
------------------------------------------------------
Accuracy             0.90 ± 0.11
Precision (AD)       0.87 ± 0.12
Recall (AD)          0.96 ± 0.08
F1-score (AD)        0.91 ± 0.10
Sensitivity          0.96 ± 0.08
Specificity          0.84 ± 0.15
------------------------------------------------------


**Leave-One-Out Cross-Validation (LOOCV)**

In [ ]:
## AAL90
AD_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_AAL90/AD"
CN_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_AAL90/CN"

X, y = load_timeseries_data(AD_DIR, CN_DIR)
results_loocv = evaluate_with_loocv(X, y)
print_formatted_results(results_loocv)


Final SVM Classification Results:
------------------------------------------------------
Metric               Mean       Std Dev
------------------------------------------------------
Accuracy             0.96 ± 0.00
Precision (AD)       0.93 ± 0.00
Recall (AD)          1.00 ± 0.00
F1-score (AD)        0.96 ± 0.00
Sensitivity          1.00 ± 0.00
Specificity          0.92 ± 0.00
------------------------------------------------------


In [ ]:
## Schaefer200
AD_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_Schaefer200/AD"
CN_DIR = "/content/drive/MyDrive/FMRI/Exp1/Timeseries_Schaefer200/CN"
X, y = load_timeseries_data(AD_DIR, CN_DIR)
results_loocv = evaluate_with_loocv(X, y)
print_formatted_results(results_loocv)


Final SVM Classification Results:
------------------------------------------------------
Metric               Mean       Std Dev
------------------------------------------------------
Accuracy             0.88 ± 0.00
Precision (AD)       0.83 ± 0.00
Recall (AD)          0.96 ± 0.00
F1-score (AD)        0.89 ± 0.00
Sensitivity          0.96 ± 0.00
Specificity          0.80 ± 0.00
------------------------------------------------------
